# Import packages 

In [4]:
import requests

url = "https://raw.githubusercontent.com/nitinmagima/endofseasonmonitoring_madagascar/refs/heads/main/utils.py"
response = requests.get(url)

# Save the file locally
with open("utils.py", "w", encoding="utf-8") as f:
    f.write(response.text)

print("utils.py downloaded successfully!")


utils.py downloaded successfully!


In [5]:
#%pip install -U "geemap[workshop]"
#!pip install earthengine-api geemap pandas matplotlib seaborn


In [6]:
import ee
import geemap
import utils as u
from IPython.display import HTML, Markdown, display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Authenticate and initialize Earth Engine

In [8]:
ee.Authenticate()

True

In [9]:
ee.Initialize(project="aerial-form-449700-h7")

# Initialize Map


In [11]:
import ipywidgets as widgets
widgets.IntSlider()


# Creating a map
m = geemap.Map(basemap='Esri.WorldTopoMap')
m.setCenter(-1.6, 12.3, 6)  # Longitude, Latitude, Zoom Level
m


Map(center=[12.3, -1.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

# NDVI and EVI Calculation Setup
## select your study area 
The Global Administrative Unit Layers (GAUL) compiles and disseminates the best available information on administrative units for all the countries in the world, providing a contribution to the standardization of the spatial dataset representing administrative units. The GAUL always maintains global layers with a unified coding system at country, first (e.g. departments), and second administrative levels (e.g. districts). Where data is available, it provides layers on a country by country basis down to third, fourth, and lowers levels.

NDVI (Normalized Difference Vegetation Index) and EVI (Enhanced Vegetation Index) are both satellite-based indices used to monitor and assess vegetation health and coverage across landscapes.

In [13]:
country_name = 'Burkina Faso'
admin_level = 'level1' #use 'level0' or 'level1'

In [14]:
roi = m.user_roi

if roi is None:
    roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}")
    roi = roi.filter(ee.Filter.eq('ADM0_NAME', country_name))
    
    # Define style parameters for visualization
    styleParams = {
        'fillColor': 'b5ffb4',
        'color': '00909F',
        'width': 1.0,
    }
    
    # Create a styled version of the ROI for visualization purposes only
    styledRoi = roi.style(**styleParams)
    
    # Add the styled ROI to the map for visualization
    m.addLayer(styledRoi, {}, country_name)

# Use 'roi' for clipping and other operations
# Do not use 'styledRoi' for operations other than visualization

# Check the type of 'roi', it should not return 'Image'
print('ROI type:', roi.getInfo()['type'])  # Should print 'FeatureCollection'

ROI type: FeatureCollection


## Load NDVI and EVI 

In [16]:
# Load the VIIRS Vegetation Indices dataset.
viirs = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')

# Directly select the EVI, EVI2, and NDVI bands from the dataset.
evi_band = 'EVI'  # 3 band Enhanced Vegetation Index
#evi2_band = 'EVI2'  # 2 band Enhanced Vegetation Index
ndvi_band = 'NDVI'  # Normalized Difference Vegetation Index

selected_bands = viirs.select([evi_band, ndvi_band])



selected_bands = selected_bands.filterBounds(roi)


selected_bands.filterBounds(roi) 

'''
This filters the selected_bands (which contain the EVI and NDVI bands) 
to include only the data that overlaps with the region of interest (roi). The filterBounds() 
function ensures that only the vegetation data for the specified region is kept, 
discarding any data outside of that region.
'''

'\nThis filters the selected_bands (which contain the EVI and NDVI bands) \nto include only the data that overlaps with the region of interest (roi). The filterBounds() \nfunction ensures that only the vegetation data for the specified region is kept, \ndiscarding any data outside of that region.\n'

# SPI Calculation Setup

sets up the Standardized Precipitation Index (SPI) calculation using the CHIRPS daily precipitation dataset in Google Earth Engine (GEE). Here’s a breakdown of what each step does:

 

### set variables

In [19]:
# set variables 

chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") 
    # Loads CHIRPS Daily Precipitation data from UCSB's Climate Hazards Group (CHG).

spimonthlyvis = {"opacity":1,"bands":["SPI"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]} 
    #Defines visualization settings for monthly SPI, using a color palette from dry (red) to wet (blue).

spi16dayvis = {"opacity":1,"bands":["SPI_16Days"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]}
    #Defines visualization for 16-day SPI with the same color scale.

### Set Time Frame 

In [21]:
#set time frame 

'''
If you want to use another period of time than the whole time span of CHIRPS data, 
change the code between ee.Date brackets (start_date & end_date) to the desired dates. Keep in mind, 
that a reduction of the time span will lead to a less accurate SPI calculation.
'''



firstimage = ee.Date(ee.List(chirps.get('date_range')).get(0))
latestimage = ee.Date(chirps.limit(1, 'system:time_start',  False).first().get('system:time_start'))

### set resolution 

In [23]:

resolution = 5550

#Sets the spatial resolution of the analysis to 5,550 meters (~5.55 km) per pixel.

### Set Time Scale Information For SPI
The SPI can be calculated based on different time scales. The scientific society usually recognizes one month as the shortest timescale for the calculation of the SPI. Shorter timescales might underly random fluctuations in precipitation. However, the SPI can also be calculated for longer timescales, like 6 months. The following settings will give you the possibility to set your own time frame for the calculation of the SPI.

Choose the number of months for the SPI. The default setting will calculate the SPI for 1 month. Setting the timestep to '6' will calculate the SPI for 6 months.

Disclaimer - The calculation works for the following quantity of months: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 48 (Need to double check this)



In [25]:

timestep = '1'


# Defines a time step for SPI calculations.
# Likely corresponds to 1 month SPI, meaning precipitation is aggregated monthly.

### Set time shift for VIIRS related SPI

The 16-day SPI product is an additional product besides the 'normal' SPI and will be calculated for the same dates as VIIRS's (NDVI and EVI) products. As the vegetation might need some time to respond to rainfall, it might be useful to apply a shift for the calculated 16-day SPI. For example: an applied shift of '-5' will cause the (16-day) SPI calculations to be started five days before the VIIRS start dates and end the calculations five days earlier than the VIIRS end dates as well. This feature might be useful when studying the response on vegetation towards rainfall. The variable "days" provides information about the observed days.

In [27]:
shift = '0'
days = '16'

## SPI calculation 
### Monthly SPI 

In [29]:
thresholdmonths = ee.Number(12)

#Create a list with a lag of one month between each list entry. Started from latest image counting backwards

timedif = (latestimage.difference(firstimage, 'month')).divide(ee.Number.parse(timestep))

In [30]:
#Creates a simple list

list = ee.List.sequence(0, timedif)
#Map the dates (beginning with the latest image) of the months ends over the list, counting backwards in time

def func_gou(month):
  zero = ee.Number(0) #Is needed to substract month
  delta = (zero.subtract(month)).multiply(ee.Number.parse(timestep)) #results in a negative counting in the list (from latest image backwards) in the steps provided by the user
  latestdate = latestimage.advance(1, 'day') #Advance one day to include the latest image (starts counting at 00:00 o'clock)
  return latestdate.advance(delta, 'month') #returns a list of dates counted from latest date backwards

timelistdate = list.map(func_gou)

In [31]:
#Sort list according to their dates

sortedtimelist = timelistdate.sort()

In [52]:
# Calculate summed CHIRPS. Just those images will be kept, whose timeframe corrensponse to the user provided number of months

def func_fxo(monthly_sum):
    # Convert timestep to ee.Number if it's not already
    timestep_num = ee.Number.parse(timestep)
    
    # Calculate start and end times
    ##Define the time range for filtering chirps 
    starttime = ee.Date(monthly_sum).advance(timestep_num.multiply(-1), 'month')
    endtime = ee.Date(monthly_sum)
    
    # Filter the CHIRPS dataset
    filteredCHIRPS = chirps.filterDate(starttime, endtime)
    
    # Clip the images to the Area of Interest
    ##ensure the precipitation data is relevant only to the target area 
    clippedCHIRPS = filteredCHIRPS.map(lambda clip: clip.clip(roi))
    
    # Calculate the number of images
    imageAmount = clippedCHIRPS.size()
    
    # Sum the images in the collection
    summedCollection = clippedCHIRPS.sum().set({
        'Used_Images': imageAmount,
        'Start_Date': ee.Date(filteredCHIRPS.first().get('system:time_start')),
        'End_Date': ee.Date(filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')),
        'system:time_start': filteredCHIRPS.first().get('system:time_start'),
        'system:time_end': filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')
    })
    
    # Calculate the observed months
    time = ee.Date(summedCollection.get('system:time_end')).difference(ee.Date(summedCollection.get('system:time_start')), 'month').round()
    
    summedImage = summedCollection.set({
        'Observed_Months': time
    })
    
    # Return the summed image only if it meets the timestep requirement
    return ee.Image(ee.Algorithms.If(time.gte(timestep_num), summedImage))

# You will need to convert this list to ee.List if it's not already, and adjust your map function accordingly
precipitationsum = ee.ImageCollection.fromImages(ee.List(timelistdate).map(func_fxo))

In [56]:
#Copy properties of CHIRPS collection to monthly collection

summedchirpscollection = ee.ImageCollection(precipitationsum.copyProperties(chirps))

In [58]:
summedchirpscollection